In [1]:
import pandas as pd
import os

In [2]:
# Lê o de-para

depara_atividades = pd.read_excel("DE-PARA.xlsx", sheet_name='ATIVIDADES')
depara_materiais = pd.read_excel("DE-PARA.xlsx", sheet_name='MATERIAIS')
depara_materiais = depara_materiais.loc[depara_materiais['STATUS'] == 'FORA DE PADRÃO']

In [3]:
# Lê o caderno de serviços

caderno_servicos = pd.read_excel("caderno de serviços.xlsx")

In [4]:
# Lê lista de materiais

lista_materiais = pd.read_excel("Lista de materiais.xlsx")

In [48]:
# Lê a ZPS60

try:
    zps60_inst = pd.read_fwf('_Mat_Serv_a_Instalar.txt', encoding = 'unicode_escape', colspecs = [(0, 24), (24, 60), (60, 140), (140, 143), (143, 154), (154, 164), (164, -1)], names = ['projeto', 'codigo', 'descricao', 'und', 'qnt', 'valor', 'tipo'])
    zps60_inst['projeto'] = zps60_inst['projeto'].replace(to_replace=r'B', value='', regex=True)
    zps60_inst['projeto'] = zps60_inst['projeto'].replace(to_replace=r'X', value='', regex=True)
    zps60_inst['projeto'] = zps60_inst['projeto'].replace(to_replace=r'Y', value='', regex=True)
    zps60_inst['projeto'] = zps60_inst['projeto'].astype(int)
except:
    zps60_inst = ''

try:
    zps60_dest = pd.read_fwf('_Mat_Serv_a_Desativar.txt', encoding = 'unicode_escape', colspecs = [(0, 24), (24, 60), (60, 140), (140, 143), (143, 154), (154, 164), (164, -1)], names = ['projeto', 'codigo', 'descricao', 'und', 'qnt', 'valor', 'tipo'])
    zps60_dest['projeto'] = zps60_dest['projeto'].replace(to_replace=r'B', value='', regex=True)
    zps60_dest['projeto'] = zps60_dest['projeto'].replace(to_replace=r'X', value='', regex=True)
    zps60_dest['projeto'] = zps60_dest['projeto'].replace(to_replace=r'Y', value='', regex=True)
    zps60_dest['projeto'] = zps60_dest['projeto'].astype(int)
except:
    zps60_dest = ''


In [49]:
# Separa materiais das atividades

if ((type(zps60_inst) != str) and (type(zps60_dest) != str)):
    atividades_inst = zps60_inst.loc[zps60_inst['tipo'] == 2]
    atividades_dest = zps60_dest.loc[zps60_dest['tipo'] == 2]
    atividades = pd.concat([atividades_inst, atividades_dest])
    atividades.reset_index(inplace=True, drop=True)
elif (type(zps60_inst) != str):
    atividades_inst = zps60_inst.loc[zps60_inst['tipo'] == 2]
    atividades = atividades_inst
elif (type(zps60_dest) != str):
    atividades_dest = zps60_dest.loc[zps60_dest['tipo'] == 2]
    atividades = atividades_dest

materiais = zps60_inst.loc[zps60_inst['tipo'] == 1]
materiais['codigo'] = materiais['codigo'].astype(int)
materiais.reset_index(inplace=True, drop=True)

C:\Users\PC1210\AppData\Local\Temp\ipykernel_11016\4039113865.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  materiais['codigo'] = materiais['codigo'].astype(int)


In [50]:
# Define valores do fator K

fator_k = pd.DataFrame([['LV', 2.7],['REDES', 1.68]], columns=['GRUPO','FATOR_K'])

In [51]:
####### TRATANDO ATIVIDADES

for count, value in enumerate(atividades.values):
    
    if value[1] in depara_atividades['cod_de'].values:
        linha = depara_atividades.loc[depara_atividades['cod_de'] == value[1]].values
        atividades.loc[count, 'codigo'] = linha[0][2]
        atividades.loc[count, 'descricao'] = linha[0][3]

    if value[2] == 'INSTALAR POSTE 9 A 14 METROS' or value[2] == 'INSTALAR POSTE 14 METROS OU SUPERIOR' or value[2] == 'INSTALAR POSTE 7 METROS' or value[2] == 'INST POSTE 10 A 16 METROS' or value[2] == 'INST POSTE 7 A 9 METROS':
        atividades = pd.concat([atividades, pd.DataFrame([[value[0], 'SIR0000001', 'CAVA NORMAL EM SOLO COMUM', 'UND', value[4], value[4]*138.7050026, 2]], columns=['projeto', 'codigo', 'descricao', 'und', 'qnt', 'valor', 'tipo'])])


atividades = atividades.dropna()
atividades = atividades.loc[~atividades['codigo'].str.contains(r'SDMT', regex=True)] # Transporte
atividades = atividades.loc[~atividades['codigo'].str.contains(r'SDET', regex=True)] # Transporte
atividades = atividades.loc[atividades['codigo'] != 'SDMMU4209SI'] # Deslocamento de turma pesada
atividades = atividades.loc[atividades['codigo'] != 'SDEMU1411II'] # Instalar ramal mono
atividades = atividades.loc[atividades['codigo'] != 'SDEMU1412II'] # Instalar ramal poli
atividades = atividades.loc[atividades['descricao'] != 'INSTALAR PLACA DA OBRA']
atividades = atividades.groupby(['projeto', 'codigo', 'descricao', 'und', 'tipo'], as_index=False).sum()

In [52]:
####### TRATANDO MATERIAIS

for count, value in enumerate(materiais['codigo'].values):
    if value in depara_materiais['DE'].values:
        linha = depara_materiais.loc[depara_materiais['DE'] == value].values
        materiais.loc[count, 'codigo'] = linha[0][3]


In [53]:
# Junta materiais e atividades

orcamentos = pd.concat([atividades,materiais])
orcamentos.reset_index(inplace=True, drop=True)

In [54]:
# Exclui projetos com código antigo

excluir = pd.concat([
    atividades.loc[atividades['codigo'].str.contains(r'BC', regex=True)],
    atividades.loc[atividades['codigo'].str.contains(r'BS', regex=True)],
    atividades.loc[atividades['codigo'].str.contains(r'BT', regex=True)]
])


excluir = excluir.groupby('projeto', as_index=False).sum()
excluir = excluir['projeto'].values

for i in excluir:
    orcamentos = (orcamentos.loc[orcamentos['projeto'] != i])

In [55]:
# Adiciona coluna de etapas

etapa = []
for count, value in enumerate(orcamentos['codigo'].values):
    if value in caderno_servicos['codigo'].values:
        linha = caderno_servicos.loc[caderno_servicos['codigo'] == value].values
        etapa.append(linha[0][5])
    else:
        etapa.append('')

orcamentos.insert(1,'etapa', etapa)

orcamentos = orcamentos.loc[(orcamentos['etapa'] != '') | (orcamentos['tipo'] == 1)]

Elaboração da planilha de diagnóstico

In [56]:
# Adiciona coluna com valor de mão de obra dos projetos

mao_de_obra = orcamentos.loc[orcamentos['tipo'] == 2]
mao_de_obra = mao_de_obra.merge(caderno_servicos[['codigo','Vlr Referência Único', 'GRUPO']], on=['codigo'], how='inner')
mao_de_obra = mao_de_obra.merge(fator_k)
mao_de_obra['Valor Total'] = mao_de_obra['qnt']*mao_de_obra['Vlr Referência Único']*mao_de_obra['FATOR_K']
mao_de_obra = mao_de_obra.groupby(['projeto'], as_index=False).sum()
mao_de_obra = mao_de_obra[['projeto', 'Valor Total']]

In [57]:
####### IDENTIFICANDO FALHA NOS ORÇAMENTOS

atividades_vs_cruz = [
    ['INSTALAR EST CRUZ SIMP ISOL PINO/PILAR', 1],
    ['INSTALAR EST CRUZ SIMP ISO PINO/PILAR LV', 1],
    ['INSTALAR EST CRUZ PARA EQUIPAMENTO LV', 2],
    ['INSTALAR EST CRUZ PARA EQUIPAMENTO', 2],
    ['INSTALAR EST CRUZ DUPLA ISOL PINO/PILAR', 2],
    ['INSTALAR EST CRUZ DUPLA 1 ANCORAGEM', 2],
    ['INSTALAR EST CRUZ DUPLA 2 ANCORAGENS', 2],
    ['INST EST CRUZ DUPLA ISOL PINO/PILAR LV', 2],
    ['INST EST CRUZ DUPLA (1 ANCORAGEM) LV', 2],
    ['INST EST CRUZ DUPLA 2 ANCORAGENS LV', 2]
]
atividades_vs_cruz = pd.DataFrame(atividades_vs_cruz, columns=['descricao', 'qnt_cruz'])

projetos = orcamentos.groupby('projeto', as_index=False).sum().projeto

orcamentos_material = orcamentos.loc[orcamentos['tipo'] == 1]
orcamentos_material = orcamentos_material.merge(lista_materiais[['codigo', 'CATEGORIA']], on=['codigo'])
orcamentos_atividade = orcamentos.loc[orcamentos['tipo'] == 2]
orcamentos_atividade_cruzetas = orcamentos_atividade.merge(atividades_vs_cruz, on=['descricao'])
orcamentos_atividade_cruzetas['qnt_cruz_atividade'] = orcamentos_atividade_cruzetas['qnt_cruz'] * orcamentos_atividade_cruzetas['qnt']
orcamentos_atividade_cruzetas = orcamentos_atividade_cruzetas.groupby('projeto', as_index=False).sum()

tabela = []

for projeto in projetos:
    orcamento_atividade = orcamentos_atividade.loc[orcamentos_atividade['projeto'] == projeto]
    orcamento_material = orcamentos_material.loc[orcamentos_material['projeto'] == projeto]

    # Divergência entre materiais vs atividades orçadas [POSTES]
    poste_atividade = orcamento_atividade.loc[orcamento_atividade['descricao'] == 'INSTALAR POSTE 9 A 14 METROS'].sum().qnt + orcamento_atividade.loc[orcamento_atividade['descricao'] == 'INSTALAR POSTE 14 METROS OU SUPERIOR'].sum().qnt
    poste_material = orcamento_material.loc[orcamento_material['CATEGORIA'] == 'POSTE'].sum().qnt

    # Divergência entre materiais vs atividades orçadas [CRUZETAS]
    cruz_atividade = orcamentos_atividade_cruzetas.loc[orcamentos_atividade_cruzetas['projeto'] == projeto].sum().qnt_cruz_atividade
    cruz_material = orcamento_material.loc[orcamento_material['CATEGORIA'] == 'CRUZETA'].sum().qnt
    
    tabela.append([projeto, poste_atividade, poste_material, cruz_atividade, cruz_material])

tabela = pd.DataFrame(tabela, columns=['projeto', 'poste_atividade', 'poste_material', 'cruzeta_atividade', 'cruzeta_material'])

tabela['diferenca_poste'] = tabela['poste_atividade'] - tabela['poste_material']
tabela['diferenca_cruzeta'] = tabela['cruzeta_atividade'] - tabela['cruzeta_material']



In [66]:
tabela.loc[tabela['diferenca_cruzeta'] > 0]

,projeto,poste_atividade,poste_material,cruzeta_atividade,cruzeta_material,diferenca_poste,diferenca_cruzeta
0,152603,21.0,21.0,54.0,48.0,0.0,6.0
1,177062,16.0,16.0,48.0,44.0,0.0,4.0
2,225599,15.0,15.0,22.0,16.0,0.0,6.0
3,262778,19.0,19.0,39.0,36.0,0.0,3.0
4,446055,8.0,8.0,24.0,16.0,0.0,8.0
7,827332,4.0,4.0,16.0,8.0,0.0,8.0
8,875614,17.0,17.0,29.0,25.0,0.0,4.0
11,927393,6.0,6.0,8.0,6.0,0.0,2.0
13,929978,4.0,4.0,6.0,4.0,0.0,2.0
14,947489,2.0,2.0,2.0,1.0,0.0,1.0


In [73]:
# Gera planilha final
writer = pd.ExcelWriter('Orçamentos.xlsx', engine='xlsxwriter')

orcamentos.to_excel(writer, sheet_name='ORÇAMENTOS', index=False)
tabela.to_excel(writer, sheet_name='DIAGNÓSTICO', index=False)

writer.save()